In [2]:
import pandas as pd
df = pd.read_csv("spam_ham_dataset.csv")
df.head(1)


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\nth...,0


In [3]:
df = df[ ["text","label_num"] ]
df.head(1)


,text,label_num
0,Subject: enron methanol ; meter # : 988291\nth...,0


In [4]:
df.iloc[0,0]

"Subject: enron methanol ; meter # : 988291\nthis is a follow up to the note i gave you on monday , 4 / 3 / 00 { preliminary\nflow data provided by daren } .\nplease override pop ' s daily volume { presently zero } to reflect daily\nactivity you can obtain from gas control .\nthis change is needed asap for economics purposes ."

In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\deniz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()

text = df.iloc[0,0]

text = text.lower()

text = text.translate(str.maketrans('','', string.punctuation))

words = text.split() # Merhaba, Python 3 -> ["Merhaba", "Python 3"] # [Merhaba,Python,3]

words = [ word for word in words if not word.isdigit()  ]

# Stem -> Kelimeleri köküne indir. (Koşuyor->Koş)
# Remove Stopwords -> The,at,is
words = [ ps.stem(word) for word in words if word not in stopwords.words("english") ]

" ".join(words)


'subject enron methanol meter follow note gave monday preliminari flow data provid daren pleas overrid pop daili volum present zero reflect daili activ obtain ga control chang need asap econom purpos'

In [7]:
def clean_text(text):
    text = text.lower()
    
    text = text.translate(str.maketrans('','', string.punctuation))
    
    words = text.split() # Merhaba, Python 3 -> ["Merhaba", "Python 3"] # [Merhaba,Python,3]
    
    words = [ word for word in words if not word.isdigit()  ]
    
    # Stem -> Kelimeleri köküne indir. (Koşuyor->Koş)
    # Remove Stopwords -> The,at,is
    words = [ ps.stem(word) for word in words if word not in stopwords.words("english") ]
    
    return " ".join(words)


In [8]:
clean_text("ödev.")

'ödev'

In [9]:
df['clean_text'] = df['text'].apply(clean_text)

In [15]:
X = df['clean_text']
y = df['label_num']
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    max_features=5000,        # En sık geçen 5000 kelime
    ngram_range=(1,2),        # Unigram ve Bigram (tek ve iki kelime) iki kelimeli kombinasyonlar
    min_df=5,                 # En az 5 dokümanda geçmesi gereken kelimeler
    max_df=0.9,               # Çok sık geçen kelimeleri filtrele (genellikle anlam taşımayanlar)
    stop_words="english"      # Dahili İngilizce stopwords
)

X_tfidf = tfidf.fit_transform(df['clean_text'])
X_tfidf


<5171x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 285176 stored elements in Compressed Sparse Row format>

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.3, random_state=42)

In [17]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB(alpha=0.1) # daha hassas , ayrım güçü yüksek sonuçlar için
model.fit(X_train,y_train)


MultinomialNB(alpha=0.1)

In [18]:
y_pred = model.predict(X_test)
y_pred

array([0, 1, 0, ..., 0, 1, 0], dtype=int64)

In [19]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.98      0.95      0.96      1121
           1       0.87      0.96      0.91       431

    accuracy                           0.95      1552
   macro avg       0.93      0.95      0.94      1552
weighted avg       0.95      0.95      0.95      1552

